In [34]:
from models.data.source import FamaFrenchData
from models.data.source import Eikon
import pickle
import numpy as np
import pandas as pd
from statsmodels.regression.linear_model import OLS
from models.stat_models.linearregression import MultiOutputLinearRegressionModel, IterativelyWeightedLeastSquares


In [35]:
data_path = r'C:\Users\serge\IdeaProjects\portfolio_management\models\data\csv\fama-french-factors'
filenames = ['Developed_5_Factors_Daily', 'Developed_MOM_Factor_Daily',
             '5_Industry_Portfolios_Daily']

PATH_API_KEYS = r'C:\Users\serge\OneDrive\reuters\apikeys.csv'

In [36]:
api = Eikon(PATH_API_KEYS)
# cons = api.get_index_constituents('.SP500', date='20240915')
cons = pickle.load(open(r'C:\Users\serge\IdeaProjects\portfolio_management\models\data\rics_of_SPX.pkl', 'rb'))

2024-10-23 18:55:18,805 P[18152] [MainThread 19140] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2024-10-23 18:55:18,807 P[18152] [MainThread 19140] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-23 18:55:18,809 P[18152] [MainThread 19140] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-23 18:55:18,809 P[18152] [MainThread 19140] Port number was not identified, cannot send any request


In [37]:
params = {
    'rics': cons,
    # 'field': ['TR.PriceClose', 'Price Close'],
    'field': ['TR.PriceClose', 'Price Close'],
    'date_field': ['TR.PriceClose.calcdate', 'Calc Date'],
    'load_path': r'C:\Users\serge\IdeaProjects\portfolio_management\models\data\csv\prices',
    # 'save_config': {'save': True, 'path': r'C:\Users\serge\IdeaProjects\portfolio_management\models\data\csv\prices' },
    # 'params': {
    #     'SDate':'2000-12-31',
    # 'EDate': '2024-08-29',
    # }
}

data = api.load_timeseries(**params, out=True)
data.set_index('Calc Date', inplace=True)


# industry and region

params = {
    'rics': cons,
    # 'fields': {'GICS Sector Name': 'TR.GICSSector'},
    'fields': ['GICS Sector Name'],
    'load_path': r'C:\Users\serge\IdeaProjects\portfolio_management\models\data\csv\fixed_factors\industry.csv',
}

[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\LIN.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPD.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_manage

In [38]:
data_industry = api.load_fixed_time_drivers(**params, out=True)
fm = FamaFrenchData(path_to_input_folder=data_path, filenames=filenames)


n_horizon = 21
ff = (1 + fm.data).rolling(n_horizon).apply(lambda x: x.prod()) - 1
returns = np.log(data / data.shift(n_horizon))[::n_horizon]

# returns.dropna(inplace=True, how='any', axis=0)
# returns.dropna(inplace=True, how='any', axis=1)
# returns.dropna(inplace=True, how='any', axis=1)

# returns = returns[returns.index > '2023-01-01']
# returns = returns.iloc[:, :100]

In [39]:
returns = returns.iloc[1:,:].dropna(axis=1, how='any')

In [40]:
indices = returns.T.index[1:]

In [41]:
data_industry.dropna(inplace=True)
indices = [x for x in indices if x in data_industry.index]
data_industry = data_industry.loc[indices]
returns = returns[indices]

In [42]:
unique_industries = list(set(list(data_industry.values.flatten())))
N_i = len(unique_industries)

In [43]:
T, M = returns.shape

In [44]:
returns

,CHRW.OQ,AJG.N,CNP.N,BA.N,WY.N,MCD.N,HD.N,SPG.N,HBAN.OQ,QRVO.OQ,...,ACGL.OQ,NDSN.OQ,AMT.N,CAG.N,TFX.N,WAT.N,INCY.OQ,LRCX.OQ,IDXX.OQ,PPG.N
Calc Date,,,,,,,,,,,,,,,,,,,,,
2001-01-30,-0.124766,-0.167733,-0.147305,-0.161628,0.024909,-0.143667,0.006817,0.090374,-0.035367,-0.255554,...,0.000000,0.174941,-0.041097,-0.090093,-0.065161,-0.161463,0.129440,0.566978,0.016902,-0.011128
2001-02-28,0.073786,-0.052272,0.116772,0.102328,0.032337,-0.001699,-0.079137,-0.025445,-0.044997,-0.647162,...,0.064539,-0.083650,-0.227969,-0.188401,0.036756,-0.075853,-0.521181,-0.173073,0.051712,0.109500
2001-03-29,-0.126978,0.031614,0.041041,-0.105897,-0.062184,-0.104227,0.003758,-0.007053,-0.072661,0.011173,...,0.003899,-0.133851,-0.407714,-0.089787,-0.035308,-0.389797,-0.167473,0.180382,-0.118115,-0.096916
2001-04-27,0.040134,-0.016068,0.100964,0.096366,0.067196,0.023135,0.120020,0.038192,0.076169,0.837344,...,-0.013318,0.161127,0.280775,0.141283,0.177244,0.101204,0.075162,0.119895,0.249096,0.146731
2001-05-28,0.087680,0.021744,-0.047150,0.027535,0.047370,0.118152,0.083332,0.026887,-0.004011,0.137803,...,0.027996,-0.092250,0.012088,-0.021957,-0.014857,0.118166,0.429314,0.046113,0.102527,0.037816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-05,-0.010899,-0.018798,-0.008873,-0.093054,0.000868,-0.093847,-0.053242,-0.014349,0.003683,-0.006927,...,0.066212,0.022190,-0.080006,0.106469,-0.035083,-0.038139,-0.070247,-0.013618,-0.070447,-0.007738
2024-05-06,0.101066,-0.004025,0.044281,-0.026503,-0.107000,0.009739,-0.044511,-0.038771,0.023257,-0.179181,...,0.023330,-0.001772,-0.062266,-0.029375,-0.075731,-0.049049,-0.021223,-0.046535,-0.085700,-0.044112
2024-06-05,0.106959,0.040643,0.037159,0.062486,-0.053871,-0.035183,-0.035778,0.060511,-0.035836,0.035030,...,0.025812,-0.169553,0.103574,-0.014010,0.069159,-0.040417,0.087581,0.049771,0.039009,0.001277


In [45]:
print(N_i)
print(T)
print(M)

11
284
237


In [81]:
# remove 1 dummy
D = pd.get_dummies(data_industry).astype(float).iloc[:,1:]
D['intercept'] = np.ones(D.shape[0])
D = D.iloc[:, ::-1]

In [82]:
D

,intercept,GICS Sector Name_Utilities,GICS Sector Name_Real Estate,GICS Sector Name_Materials,GICS Sector Name_Information Technology,GICS Sector Name_Industrials,GICS Sector Name_Health Care,GICS Sector Name_Financials,GICS Sector Name_Energy,GICS Sector Name_Consumer Staples,GICS Sector Name_Consumer Discretionary
Instrument,,,,,,,,,,,
CHRW.OQ,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AJG.N,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
CNP.N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BA.N,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
WY.N,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
WAT.N,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
INCY.OQ,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
LRCX.OQ,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
Y = returns

In [84]:
# _industry_map = {j:i for i,j in zip(range(n_industry), unique_industries)}
# returns = returns.reset_index().rename({'Calc Date': 'time'})
# design_matrix = returns.T.join(data_industry, how='left')
# design_matrix.dropna(inplace=True, how='any', axis=0)
# design_matrix = design_matrix.replace({'GICS Sector Name': _industry_map})

In [85]:
Y.shape

(284, 237)

In [86]:
D.shape

(237, 11)

In [87]:
model = MultiOutputLinearRegressionModel(x=D.astype(float),
                                 y=Y.T,
                                 method='ols')

model.fit()

In [89]:
model.betas[:,0]

array([ 0.00022734,  0.05851961, -0.14744827, -0.00841246, -0.05479188,
        0.17656935, -0.03668805, -0.09217377, -0.04441366, -0.11081272,
       -0.05051471,  0.01917949])

In [62]:
returns.iloc[:,0].shape

(284,)

In [90]:
m = OLS(endog=returns.iloc[0,:], exog=D, hasconst=True)
res = m.fit()

In [97]:
res.params

intercept                                  0.041274
GICS Sector Name_Utilities                -0.147448
GICS Sector Name_Real Estate              -0.008412
GICS Sector Name_Materials                -0.054792
GICS Sector Name_Information Technology    0.176569
GICS Sector Name_Industrials              -0.036688
GICS Sector Name_Health Care              -0.092174
GICS Sector Name_Financials               -0.044414
GICS Sector Name_Energy                   -0.110813
GICS Sector Name_Consumer Staples         -0.050515
GICS Sector Name_Consumer Discretionary    0.019179
dtype: float64

GICS Sector Name_Communication Services    0.041274
dtype: float64

In [10]:
X = design_matrix.join(indicators, how='left')

In [12]:
X.T.head()

,POOL.OQ,CHRW.OQ,AJG.N,CNP.N,BA.N,WY.N,MCD.N,HD.N,SPG.N,HBAN.OQ,...,ACGL.OQ,NDSN.OQ,AMT.N,CAG.N,TFX.N,WAT.N,INCY.OQ,LRCX.OQ,IDXX.OQ,PPG.N
0,0.0683,-0.124766,-0.167733,-0.147305,-0.161628,0.024909,-0.143667,0.006817,0.090374,-0.035367,...,0.0,0.174941,-0.041097,-0.090093,-0.065161,-0.161463,0.12944,0.566978,0.016902,-0.011128
1,0.032477,0.073786,-0.052272,0.116772,0.102328,0.032337,-0.001699,-0.079137,-0.025445,-0.044997,...,0.064539,-0.08365,-0.227969,-0.188401,0.036756,-0.075853,-0.521181,-0.173073,0.051712,0.1095
2,-0.064037,-0.126978,0.031614,0.041041,-0.105897,-0.062184,-0.104227,0.003758,-0.007053,-0.072661,...,0.003899,-0.133851,-0.407714,-0.089787,-0.035308,-0.389797,-0.167473,0.180382,-0.118115,-0.096916
3,0.01314,0.040134,-0.016068,0.100964,0.096366,0.067196,0.023135,0.12002,0.038192,0.076169,...,-0.013318,0.161127,0.280775,0.141283,0.177244,0.101204,0.075162,0.119895,0.249096,0.146731
4,0.105044,0.08768,0.021744,-0.04715,0.027535,0.04737,0.118152,0.083332,0.026887,-0.004011,...,0.027996,-0.09225,0.012088,-0.021957,-0.014857,0.118166,0.429314,0.046113,0.102527,0.037816


In [ ]:
model = OLS(X.iloc[0,:], X.iloc[0,:][_cols_ind], hasconst=False)

In [ ]:
X.iloc[0,:][_cols_ind]